# Detection of subgenomic RNA sites for COVID-19
This project is based on [paper](https://www.nature.com/articles/s41467-020-19883-7#Sec12) which looks at open reading frames (ORF) and their potential role in COVID-19. The code here is a fork of the original [repo](https://github.com/achamings/SARS-CoV-2-leader) which filters whole genome sequence (WGS) samples on the sequence motif of GTAGATCTGTTCTCT. The resulting mapped sequences are then used to detect subgenomic RNA sites through the relative depth of coverage at each site of interest as defined in the paper.

## Installation/Setup
Please ensure you've followed instructions in `./README.md` to install the conda environment which this is based on. It contains the required libraries for also running this jupyter notebook. If you follow the default set up then ensure your Jupyter server is pointing to `./venv/bin/python`.

## Libraries
Import of libraries. These are used for data processing and type hinting.

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
from concurrent.futures import ThreadPoolExecutor # For parallel processing
import subprocess
import math
from pathlib import Path

Quick check to ensure the currect directory is the root of the project. This should be the github repo.

In [ ]:
%pwd

## Variables
A `./.env` file in the project root can be used to store variables. This is .gitignored so paths are not commited to the repo. Please see `./notebooks/covid19_leader.env.template` for an example. This file also can be skipped and then the default values will be used.

In [ ]:
load_dotenv(find_dotenv())
INPUT_DIR = os.getenv('INPUT_DIR', "./input")
SARS_COV_2_LEADER_PROGRAM_PATH = os.getenv('SARS_COV_2_LEADER_PROGRAM_PATH', "./scripts/sars_cov2_leader.sh")
OUTPUT_DIR = os.getenv('OUT_DIR', "./output")
THREADS = os.getenv('THREADS', 12)
EXECUTE_COMMANDS = os.getenv('EXECUTE_COMMANDS', True)

PROJECT_PATH = os.getcwd()

## Functions and calls
The notebook is organized into a defined function for utility then a call associated with it. Additional informational code may be also included.

### Get bam files
Gathers all bam files in the input directory and returns a list of the full file paths.

In [ ]:
def get_bam_files(input_dir:Path, bam_extension:str=".bam") -> list[Path]:
    bam_files:list[Path] = []
    for here, dirs, files in os.walk(input_dir, topdown=True):
        for file in files:
            if file.endswith(bam_extension):
                bam_files.append(os.path.abspath(os.path.join(here, file)))
    return bam_files

In [ ]:
bam_files = get_bam_files(INPUT_DIR) 

Sanity check on expected inputs and peek at first few results

In [ ]:
print(f"Length of bam_files: {len(bam_files)}\nFirst 10: {bam_files[:10]}\n")

### Find leaders in bam
This wrapper runs the original script `./scripts/sars_cov2_leader.sh` against each bam file found. This will filter the mapped reads for reads with the leader sequence which will be accessed later.

As the original script was not designed to run in batch mode this handles that and requires that the output file is unique to each sample as their is no lock on writing to the output file. The script also assumes that it operates in the current directory so will place its output in `./`. This is not ideal as if the script fails temporary files will be left behind, however, the decision was made to not adjust the original code in case of updates to the original script.

In [84]:
def find_leaders_in_bam(bam_files: list[Path], output:Path, program_path:Path=SARS_COV_2_LEADER_PROGRAM_PATH, reference_name:str='MN908947.3', quality:int=30) -> list[Path]:
    output_folders: list[Path] = []
    commands: list[str] = []
    process_threads: int = 1 # Set to 1 because each process runs really fast so figure its more optimal to run 1 per thread then pooling more threads to 1 
    try:    
        for bam_file in bam_files:
            expected_output_folder:Path = os.path.join(output, f"{os.path.basename(bam_file)[:-4]}_leader_data")
            out_leaders_txt:Path = os.path.join(output, os.path.basename(bam_file))[:-4] + ".leaders.txt"
            if not os.path.isdir(expected_output_folder):    
                command:str = f"" \
                    f"{program_path} -i {bam_file} -r {reference_name} -q {quality} -t {process_threads} -o {out_leaders_txt};" \
                    f"mv {os.path.basename(bam_file)[:-4]}_leader_data {expected_output_folder};" # Dislike this as if it fails it leaves temp files everywhere. I'm avoiding modifying the original code though so leaving this as is
                commands.append(command)
        with ThreadPoolExecutor(max_workers=math.floor(THREADS/process_threads)) as executor:
            for i, command in enumerate(commands):
                if EXECUTE_COMMANDS:
                    executor.submit(subprocess.run, command, shell=True)
                    print(f"{i}/{len(commands)} Ran command: {command}", end="\r")
    except Exception as e:
        print(f"Error: {e}\n"\
            f"Command: {command if 'command' in locals() else None}\n"\
            f"Error processing: {bam_file if 'bam_file' in locals() else None}")
    return output_folders

In [ ]:
leader_folder = find_leaders_in_bam(bam_files, OUTPUT_DIR)

Scanning for bam files for output instead of using `leader_folder` parsing in case user starts with previous output or adds completed output

In [ ]:
output_bam_files = get_bam_files(OUTPUT_DIR)

### Calculate depth on the leader data
This runs the `samtools depth` command on all the filtered bam files so that the coverage can be extracted for sites of interest.

In [85]:
def calculate_depth_on_leader_data(bam_files:list[Path], output:Path) -> list[Path]:
    calculated_depth_files:list[Path] = []
    commands:str = []
    try:
        for bam_file in bam_files:
            output_depth_file:Path = os.path.join(output, os.path.basename(bam_file))[:-4] + ".depth.txt"
            calculated_depth_files.append(output_depth_file)
            if not os.path.isfile(output_depth_file):
                command:str = f"" \
                    f"samtools depth {bam_file} > {output_depth_file}; " 
                commands.append(command)
        with ThreadPoolExecutor(max_workers=THREADS) as executor:
            for i, command in enumerate(commands):
                print(f"\n{command}")
                if EXECUTE_COMMANDS:
                    executor.submit(subprocess.run, command, shell=True)
                    print(f"{i}/{len(commands)}\ Depth calculated: {bam_file}", end="\r")
    except Exception as e:
        print(f"Error: {e}\n"\
            f"Command: {command if 'command' in locals() else None}\n"\
            f"Error processing: {bam_file if 'bam_file' in locals() else None}")
    return calculated_depth_files

In [86]:
depth_files = calculate_depth_on_leader_data(output_bam_files, OUTPUT_DIR) 

### Scan for depth files
Gathers the depth files, is equivalent to running `calculate_depth_on_leader_data` but allows for inclusion of data from previous runs or if you lack inputs to the function.

In [ ]:
def scan_for_depth_files(output:Path) -> list[Path]:
    depth_files:list[Path] = []
    for root, dirs, files in os.walk(output):
        for file in files:
            if file.endswith(".depth.txt"):
                depth_files.append(os.path.abspath(os.path.join(root, file)))
    return depth_files

In [75]:
depth_files = scan_for_depth_files(OUTPUT_DIR)

### Sites of interest
The sites of interest from the [paper](https://www.nature.com/articles/s41467-020-19883-7#Sec12). An attempt was also done to detect peeks in the depth data by considering previous position, minimum depth, and change from previous position. This did lead to comparable sites of interest but also included spurious sites of interest, for example positions within the read length of another position due to deletions or that the change from previous position was too small or small mismappings which skewed positions by a handful of positions. A generic solution would of course be ideal but this is a project for the future.

In [ ]:
sites_from_paper = [55, 21552, 25385, 26237, 26469, 27041, 27388, 27644, 27884, 28256, 29530]

### Parse depth on sites of interest
Retrieve the coverage of positions in the sites of interest.

In [80]:
def parse_depth_on_sites_of_interest(depth_files: list[Path], sites_of_interest: list[int], output: Path):
    with open(output, "w+") as out_file:
        for depth_file in depth_files:
            sample_name:str = os.path.basename(depth_file).split(".")[0]
            with open(depth_file, "r") as depth_file_stream:
                print(f"#sample_name\tposition\tcount", file=out_file)
                for line in depth_file_stream:
                    position = int(line.strip().split("\t")[1])
                    depth = int(line.strip().split("\t")[2])
                    if position in sites_of_interest:
                        print(f"{sample_name}\t{position}\t{depth}", file=out_file)

In [81]:
parse_depth_on_sites_of_interest(depth_files, sites_from_paper, f"{OUTPUT_DIR}/COVID_leader_splice_sites.tsv")

### Make proportional samples
Calculates the proportion on sites of interest for each sample.

In [82]:
def make_proportional_samples(parsed_depth_file:Path, proportional_file:Path):
    sample_count:dict = {}
    with open(parsed_depth_file, "r") as f:
        for line in f:
            line:str = line.strip()
            if not line.startswith("#"):    
                line = line.split("\t")
                try:
                    sample:str = line[0]
                    pos:int = int(line[1])
                    counts:int = int(line[2])
                    if sample not in sample_count:
                        sample_count[sample] = {}
                    sample_count[sample][pos] = counts
                except Exception:
                    print(line)
    with open(proportional_file, "w+") as out_file:
        print(f"#sample_name\tposition\tproportion\tcount", file=out_file)
        for key in sample_count:
            total:int = 0
            total:int = sum(sample_count[key].values())
            for pos in sample_count[key]:
                print(f"{key}\t{pos}\t{sample_count[key][pos]/total}\t{sample_count[key][pos]}", file=out_file)


In [83]:
make_proportional_samples(f"{OUTPUT_DIR}/COVID_leader_splice_sites.tsv", f"{OUTPUT_DIR}/COVID_leader_splice_sites.proportional.tsv")